# Visibility Tutorial

## Import xradio

In [1]:
import os, pprint
from importlib.metadata import version

try:
    os.system("pip install --upgrade xradio")

    import xradio

    print("Using xradio version", version("xradio"))

except ImportError as exc:
    print(f"Could not import xradio: {exc}")

Using xradio version 0.0.19


## Download example MSv2

## Preparation

In [2]:
import graphviper

graphviper.utils.data.download(file="Antennae_North.cal.lsrk.split.ms")

[2024-02-21 08:22:44,788]     INFO  graphviper:  File exists: Antennae_North.cal.lsrk.split.ms 


## Processing Set

## Convert MSv2 => Processing Set (PS)

In [3]:
from xradio.vis.convert_msv2_to_processing_set import convert_msv2_to_processing_set

msv2_name = "Antennae_North.cal.lsrk.split.ms"
partition_scheme = "ddi_intent_field"
convert_out = "Antennae_North.cal.lsrk.split.vis.zarr"

convert_msv2_to_processing_set(
    in_file=msv2_name,
    out_file=convert_out,
    partition_scheme=partition_scheme,
    overwrite=True,
)

## Lazy read PS

In [4]:
from xradio.vis.read_processing_set import read_processing_set

intents = ["OBSERVE_TARGET#ON_SOURCE"]
ps = read_processing_set(convert_out, intents=intents)

In [5]:
ps.summary()

,name,ddi,intent,field_id,field_name,start_frequency,end_frequency
0,Antennae_North.cal.lsrk.split_ddi_0_intent_OBS...,0,OBSERVE_TARGET#ON_SOURCE,1,NGC4038 - Antennae North,3.439281e+11,3.440067e+11
1,Antennae_North.cal.lsrk.split_ddi_0_intent_OBS...,0,OBSERVE_TARGET#ON_SOURCE,0,NGC4038 - Antennae North,3.439281e+11,3.440067e+11
2,Antennae_North.cal.lsrk.split_ddi_0_intent_OBS...,0,OBSERVE_TARGET#ON_SOURCE,2,NGC4038 - Antennae North,3.439281e+11,3.440067e+11


## PS Structure

A processing set is simply a dictionary of MSv4s (one per observation, field, intent, spectral window - polarization...):

In [6]:
len(ps)

3

In [7]:
ps.keys()

dict_keys(['Antennae_North.cal.lsrk.split_ddi_0_intent_OBSERVE_TARGET#ON_SOURCE_field_id_1', 'Antennae_North.cal.lsrk.split_ddi_0_intent_OBSERVE_TARGET#ON_SOURCE_field_id_0', 'Antennae_North.cal.lsrk.split_ddi_0_intent_OBSERVE_TARGET#ON_SOURCE_field_id_2'])

## MSv4


## Main dataset

We can take one of the items of the Processing Set to look into the contents of that MSv4. Every MSv4 represents the data as an xarray dataset, similarly as in earlier CNGI prototypes. The data variables (visibilities, weights, flags, etc.) can be manipulated and used in computations using the xarray API.

In [8]:
main_xds = ps[
    "Antennae_North.cal.lsrk.split_ddi_0_intent_OBSERVE_TARGET#ON_SOURCE_field_id_1"
]

In [9]:
main_xds

<xarray.Dataset>
Dimensions:                     (time: 35, baseline_id: 64, frequency: 8,
                                 polarization: 2, uvw_label: 3)
Coordinates:
    baseline_antenna1_id        (baseline_id) int32 dask.array<chunksize=(64,), meta=np.ndarray>
    baseline_antenna2_id        (baseline_id) int32 dask.array<chunksize=(64,), meta=np.ndarray>
  * baseline_id                 (baseline_id) int64 0 1 2 3 4 ... 59 60 61 62 63
  * frequency                   (frequency) float64 3.439e+11 ... 3.44e+11
  * polarization                (polarization) <U2 'XX' 'YY'
  * time                        (time) float64 1.307e+09 1.307e+09 ... 1.307e+09
  * uvw_label                   (uvw_label) <U1 'u' 'v' 'w'
Data variables:
    EFFECTIVE_INTEGRATION_TIME  (time, baseline_id) float64 dask.array<chunksize=(35, 64), meta=np.ndarray>
    FLAG                        (time, baseline_id, frequency, polarization) bool dask.array<chunksize=(35, 64, 8, 2), meta=np.ndarray>
    TIME_CENTROID               (time, baseline_id) float64 dask.array<chunksize=(35, 64), meta=np.ndarray>
    UVW                         (time, baseline_id, uvw_label) float64 dask.array<chunksize=(35, 64, 3), meta=np.ndarray>
    VISIBILITY                  (time, baseline_id, frequency, polarization) complex64 dask.array<chunksize=(35, 64, 8, 2), meta=np.ndarray>
    WEIGHT                      (time, baseline_id, frequency, polarization) float32 dask.array<chunksize=(35, 64, 8, 2), meta=np.ndarray>
Attributes:
    data_groups:   {'base': {'data': 'VISIBILITY', 'flag': 'FLAG', 'uvw': 'UV...
    ddi:           0
    intent:        OBSERVE_TARGET#ON_SOURCE
    antenna_xds:   <xarray.Dataset>\nDimensions:        (antenna_id: 15, xyz_...
    weather_xds:   <xarray.Dataset>\nDimensions:         (station_id: 2, time...
    pointing_xds:  <xarray.Dataset>\nDimensions:  ()\nData variables:\n    *e...

#### Coordinates

In [10]:
main_xds.polarization

<xarray.DataArray 'polarization' (polarization: 2)>
array(['XX', 'YY'], dtype='<U2')
Coordinates:
  * polarization  (polarization) <U2 'XX' 'YY'

In [11]:
main_xds.uvw_label

<xarray.DataArray 'uvw_label' (uvw_label: 3)>
array(['u', 'v', 'w'], dtype='<U1')
Coordinates:
  * uvw_label  (uvw_label) <U1 'u' 'v' 'w'

In [12]:
main_xds.coords["baseline_id"]

<xarray.DataArray 'baseline_id' (baseline_id: 64)>
array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
       54, 55, 56, 57, 58, 59, 60, 61, 62, 63])
Coordinates:
    baseline_antenna1_id  (baseline_id) int32 dask.array<chunksize=(64,), meta=np.ndarray>
    baseline_antenna2_id  (baseline_id) int32 dask.array<chunksize=(64,), meta=np.ndarray>
  * baseline_id           (baseline_id) int64 0 1 2 3 4 5 ... 58 59 60 61 62 63

In [13]:
main_xds.time

<xarray.DataArray 'time' (time: 35)>
array([1.306547e+09, 1.306547e+09, 1.306547e+09, 1.306547e+09, 1.306547e+09,
       1.306549e+09, 1.306549e+09, 1.306549e+09, 1.306549e+09, 1.306549e+09,
       1.306550e+09, 1.306550e+09, 1.306550e+09, 1.306550e+09, 1.306550e+09,
       1.307137e+09, 1.307137e+09, 1.307137e+09, 1.307137e+09, 1.307137e+09,
       1.307138e+09, 1.307138e+09, 1.307138e+09, 1.307138e+09, 1.307138e+09,
       1.307139e+09, 1.307139e+09, 1.307139e+09, 1.307139e+09, 1.307139e+09,
       1.307148e+09, 1.307148e+09, 1.307148e+09, 1.307148e+09, 1.307148e+09])
Coordinates:
  * time     (time) float64 1.307e+09 1.307e+09 ... 1.307e+09 1.307e+09
Attributes:
    effective_integration_time:  EFFECTIVE_INTEGRATION_TIME
    format:                      unix
    integration_time:            {'attrs': {'type': 'quantity', 'units': ['s'...
    scale:                       UTC
    type:                        time
    units:                       ['s']

#### Data vars

In [14]:
main_xds.VISIBILITY

<xarray.DataArray 'VISIBILITY' (time: 35, baseline_id: 64, frequency: 8,
                                polarization: 2)>
dask.array<open_dataset-VISIBILITY, shape=(35, 64, 8, 2), dtype=complex64, chunksize=(35, 64, 8, 2), chunktype=numpy.ndarray>
Coordinates:
    baseline_antenna1_id  (baseline_id) int32 dask.array<chunksize=(64,), meta=np.ndarray>
    baseline_antenna2_id  (baseline_id) int32 dask.array<chunksize=(64,), meta=np.ndarray>
  * baseline_id           (baseline_id) int64 0 1 2 3 4 5 ... 58 59 60 61 62 63
  * frequency             (frequency) float64 3.439e+11 3.439e+11 ... 3.44e+11
  * polarization          (polarization) <U2 'XX' 'YY'
  * time                  (time) float64 1.307e+09 1.307e+09 ... 1.307e+09
Attributes:
    field_info:  {'code': 'none', 'delay_direction': {'attrs': {'frame': 'FK5...
    type:        quanta
    units:       ['unkown']

In [15]:
main_xds.FLAG

<xarray.DataArray 'FLAG' (time: 35, baseline_id: 64, frequency: 8,
                          polarization: 2)>
dask.array<open_dataset-FLAG, shape=(35, 64, 8, 2), dtype=bool, chunksize=(35, 64, 8, 2), chunktype=numpy.ndarray>
Coordinates:
    baseline_antenna1_id  (baseline_id) int32 dask.array<chunksize=(64,), meta=np.ndarray>
    baseline_antenna2_id  (baseline_id) int32 dask.array<chunksize=(64,), meta=np.ndarray>
  * baseline_id           (baseline_id) int64 0 1 2 3 4 5 ... 58 59 60 61 62 63
  * frequency             (frequency) float64 3.439e+11 3.439e+11 ... 3.44e+11
  * polarization          (polarization) <U2 'XX' 'YY'
  * time                  (time) float64 1.307e+09 1.307e+09 ... 1.307e+09

In [16]:
main_xds.VISIBILITY.max()

<xarray.DataArray 'VISIBILITY' ()>
dask.array<_nanmax_skip-aggregate, shape=(), dtype=complex64, chunksize=(), chunktype=numpy.ndarray>

In [17]:
main_xds.VISIBILITY.max().compute()
# main_xds.VISIBILITY.max().values

<xarray.DataArray 'VISIBILITY' ()>
array(7.9943047+7.3078017j, dtype=complex64)

## Metadata

The MS metadata can be found in the attributes of the main_xds. Metadata is stored in differente ways:
- in additional xarray (sub)datasets, "sub-xds"
- in attributes of coordinates and data variables
- in Python dictionaries.

An example of sub-xds is the antenna dataset. And example of dictionary is the Field info dict.

### Metadata in sub-xds. Antenna dataset

The MSv4 has xarray datasets in its attributes that represent metadata where n-dimensional arrays is included. This would be the equivalent to subtables of the MSv2. Let's look into the antenna sub-xds:


In [18]:
ant_xds = main_xds.attrs["antenna_xds"]

In [19]:
ant_xds

<xarray.Dataset>
Dimensions:        (antenna_id: 15, xyz_label: 3)
Coordinates:
  * antenna_id     (antenna_id) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14
    mount          (antenna_id) <U6 dask.array<chunksize=(15,), meta=np.ndarray>
    name           (antenna_id) <U4 dask.array<chunksize=(15,), meta=np.ndarray>
    station        (antenna_id) <U4 dask.array<chunksize=(15,), meta=np.ndarray>
    type           (antenna_id) <U12 dask.array<chunksize=(15,), meta=np.ndarray>
  * xyz_label      (xyz_label) <U1 'x' 'y' 'z'
Data variables:
    DISH_DIAMETER  (antenna_id) float64 dask.array<chunksize=(15,), meta=np.ndarray>
    FEED_OFFSET    (antenna_id, xyz_label) float64 dask.array<chunksize=(15, 3), meta=np.ndarray>
    POSITION       (antenna_id, xyz_label) float64 dask.array<chunksize=(15, 3), meta=np.ndarray>

As an xarray dataset, the antenna sub-xds can be used via the same API as the main xds.

In [20]:
ant_xds.POSITION  # .values to load and see them

<xarray.DataArray 'POSITION' (antenna_id: 15, xyz_label: 3)>
dask.array<open_dataset-POSITION, shape=(15, 3), dtype=float64, chunksize=(15, 3), chunktype=numpy.ndarray>
Coordinates:
  * antenna_id  (antenna_id) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14
    mount       (antenna_id) <U6 dask.array<chunksize=(15,), meta=np.ndarray>
    name        (antenna_id) <U4 dask.array<chunksize=(15,), meta=np.ndarray>
    station     (antenna_id) <U4 dask.array<chunksize=(15,), meta=np.ndarray>
    type        (antenna_id) <U12 dask.array<chunksize=(15,), meta=np.ndarray>
  * xyz_label   (xyz_label) <U1 'x' 'y' 'z'
Attributes:
    ellipsoid:  GRS80
    type:       earth_location
    units:      ['m', 'm', 'm']

### Attributes of Data Arrays and Coordinates. Quantities and Measures

All data variables and coordinates can have quantity and measures information in their attributes section along with other relevant metadata. These measures are specified as dictionaries in the attribute of the data variable or coordinate, with keys `units` and `type` in addition to other keys depending on the type of quantity. The naming conventions are based on `astropy`. For example a quantity of casacore/`position` type, such as the antenna positions, is a quantity with `type: "earth_location"`

For reference, this is the list of measures in the current Processing Set/MSv4 spec:
https://docs.google.com/spreadsheets/d/14a6qMap9M5r_vjpLnaBKxsR9TF4azN5LVdOxLacOX-s/edit#gid=1504318014, with naming conventions based on astropy. For example, a casacore `direction` is a `sky_coord`.



#### Time coordinate
The time coordinate is a time measure (keys: `type`, `units`, `time_scale`, `format`) but also contains for example `integration_time` which is a quantity.

In [21]:
main_xds.time

<xarray.DataArray 'time' (time: 35)>
array([1.306547e+09, 1.306547e+09, 1.306547e+09, 1.306547e+09, 1.306547e+09,
       1.306549e+09, 1.306549e+09, 1.306549e+09, 1.306549e+09, 1.306549e+09,
       1.306550e+09, 1.306550e+09, 1.306550e+09, 1.306550e+09, 1.306550e+09,
       1.307137e+09, 1.307137e+09, 1.307137e+09, 1.307137e+09, 1.307137e+09,
       1.307138e+09, 1.307138e+09, 1.307138e+09, 1.307138e+09, 1.307138e+09,
       1.307139e+09, 1.307139e+09, 1.307139e+09, 1.307139e+09, 1.307139e+09,
       1.307148e+09, 1.307148e+09, 1.307148e+09, 1.307148e+09, 1.307148e+09])
Coordinates:
  * time     (time) float64 1.307e+09 1.307e+09 ... 1.307e+09 1.307e+09
Attributes:
    effective_integration_time:  EFFECTIVE_INTEGRATION_TIME
    format:                      unix
    integration_time:            {'attrs': {'type': 'quantity', 'units': ['s'...
    scale:                       UTC
    type:                        time
    units:                       ['s']

##### Quantities and measures that are not xarray

When a quantity or a measure is not an xarray, it is specificed as a dictionary with a format based on xarray's [xarray.DataArray.from_dict()](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.from_dict.html) and it has the following keys:
`{"dims": ..., "data": ..., "attrs": quantity/measures_dict}`. The `integration_time` attribute included in the  attributes of the time coordinate is an example:

In [22]:
pprint.pprint(main_xds.time.attrs)

{'effective_integration_time': 'EFFECTIVE_INTEGRATION_TIME',
 'format': 'unix',
 'integration_time': {'attrs': {'type': 'quantity', 'units': ['s']},
                      'data': 6.048,
                      'dims': ''},
 'scale': 'UTC',
 'type': 'time',
 'units': ['s']}


#### Frequency coordinate

The `frequency` coordinate is a `spectral_coord` measure and as such has the following keys in its attributes: `type`, `units`, and `frame`. In addition, the attributes contain the `channel_width`, `spectral_window_name`, and `reference_frequency`.

Any metadata that is a quantity or measure (non-id numbers) is placed in the relevant measures or quantity dictionary.

In [23]:
main_xds.frequency

<xarray.DataArray 'frequency' (frequency: 8)>
array([3.439281e+11, 3.439393e+11, 3.439506e+11, 3.439618e+11, 3.439730e+11,
       3.439843e+11, 3.439955e+11, 3.440067e+11])
Coordinates:
  * frequency  (frequency) float64 3.439e+11 3.439e+11 ... 3.44e+11 3.44e+11
Attributes:
    channel_width:         {'attrs': {'type': 'quantity', 'units': ['Hz']}, '...
    frame:                 LSRK
    reference_frequency:   {'attrs': {'frame': 'LSRK', 'type': 'spectral_coor...
    spectral_window_name:  
    spw_id:                0
    type:                  spectral_coord
    units:                 ['Hz']

In the frequency coordinate we have example of:
- quantity given as a dict: `channel_width`
- measure given as a dict: `reference_frequency` (a `spectral_coord` ~= casacore/frequency)

In [24]:
pprint.pprint(main_xds.frequency.attrs)

{'channel_width': {'attrs': {'type': 'quantity', 'units': ['Hz']},
                   'data': 11231488.981445312,
                   'dims': ''},
 'frame': 'LSRK',
 'reference_frequency': {'attrs': {'frame': 'LSRK',
                                   'type': 'spectral_coord',
                                   'units': ['Hz']},
                         'data': 343928096685.9587,
                         'dims': ''},
 'spectral_window_name': '',
 'spw_id': 0,
 'type': 'spectral_coord',
 'units': ['Hz']}


### Metadata in dicts. Field info.

The MSv4 also allows for info dictionaries in the attribute section of the dataset. This is used when no n-dimensional data is required. The relevant measures metadata is included, similarly as with coordinates and data variables (when non-id) in xarray datasets.

An example is the field_info where the delay_direction, phase_direction, and reference_direction are stored as `sky_coord` measures (keys: `type`, `units`, `reference_frame`).

In [26]:
pprint.pprint(main_xds.VISIBILITY.field_info)

{'code': 'none',
 'delay_direction': {'attrs': {'frame': 'FK5',
                               'type': 'sky_coord',
                               'units': ['rad', 'rad']},
                     'data': [3.149807242890337, -0.3292996087690852],
                     'dims': ''},
 'field_id': 1,
 'name': 'NGC4038 - Antennae North',
 'phase_direction': {'attrs': {'frame': 'FK5',
                               'type': 'sky_coord',
                               'units': ['rad', 'rad']},
                     'data': [3.149807242890337, -0.3292996087690852],
                     'dims': ''},
 'reference_direction': {'attrs': {'frame': 'FK5',
                                   'type': 'sky_coord',
                                   'units': ['rad', 'rad']},
                         'data': [3.149807242890337, -0.3292996087690852],
                         'dims': ''}}


## Selection examples

One can use the usual selection functionality of xarray with all arrays, the main dataset and all sub datasets. For example, selection by labels, `sel()`:

In [27]:
sel_xds = main_xds.sel(frequency=slice(3.43939e11, 3.4397e11))
sel_xds.frequency

<xarray.DataArray 'frequency' (frequency: 3)>
array([3.439393e+11, 3.439506e+11, 3.439618e+11])
Coordinates:
  * frequency  (frequency) float64 3.439e+11 3.44e+11 3.44e+11
Attributes:
    channel_width:         {'attrs': {'type': 'quantity', 'units': ['Hz']}, '...
    frame:                 LSRK
    reference_frequency:   {'attrs': {'frame': 'LSRK', 'type': 'spectral_coor...
    spectral_window_name:  
    spw_id:                0
    type:                  spectral_coord
    units:                 ['Hz']

Or selection by indices, `isel()`

In [28]:
isel_xds = main_xds.isel(frequency=slice(1, 4))
isel_xds.frequency

<xarray.DataArray 'frequency' (frequency: 3)>
array([3.439393e+11, 3.439506e+11, 3.439618e+11])
Coordinates:
  * frequency  (frequency) float64 3.439e+11 3.44e+11 3.44e+11
Attributes:
    channel_width:         {'attrs': {'type': 'quantity', 'units': ['Hz']}, '...
    frame:                 LSRK
    reference_frequency:   {'attrs': {'frame': 'LSRK', 'type': 'spectral_coor...
    spectral_window_name:  
    spw_id:                0
    type:                  spectral_coord
    units:                 ['Hz']

In [29]:
sel_xds.equals(isel_xds)

True

In [30]:
sel_xds.identical(isel_xds)

True